<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
Brief explanation on our Booter database analysis methodology: <br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

# Analysis of: pandabooter2
#### File originally retrieved from: http://pastebin.com/0Vddej44

## * Libraries that I use to analyse data

In [2]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 0: Reading an input Booter database file<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [3]:
dumpfile='pandabooter2.sql'

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 1: Adaptation to our Booter database schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 1. Reading the first 'N' (100) lines of the input Booter database file

In [7]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

CREATE TABLE `email_templates` (
`id` int(5) NOT NULL AUTO_INCREMENT,
`name` varchar(200) NOT NULL,
`subject` varchar(255) NOT NULL,
`help` text,
`body` text,
PRIMARY KEY (`id`)
) ENGINE=MyISAM AUTO_INCREMENT=15 DEFAULT CHARSET=utf8;
INSERT INTO `email_templates` (`id`, `name`, `subject`, `help`, `body`) VALUES
(1, 'Registration Email', 'Please verify your email', 'This template is used to send Registration Verification Email, when Configuration->Registration Verification is set to YES', '<div align="center">\n<table cellspacing="5" cellpadding="5" border="0" width="600" style="background: none repeat scroll 0% 0% rgb(244, 244, 244); border: 1px solid rgb(102, 102, 102);">\n    <tbody>\n        <tr>\n            <th style="background-color: rgb(204, 204, 204);">Welcome [NAME]! Thanks for registering.</th>\n        </tr>\n        <tr>\n            <td valign="top" style="text-align: left;">Hello,<br/>\n            <br/>\n            You\'re now a member of [SITE_NAME].<br/>\n           

## 2. Listing tables that have content inserted into the dump file

In [8]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [10]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

1 email_templates
2 gateways
3 getshells
4 logs
5 memberships
6 news
7 postshells
8 settings
9 users


## How the list above looks like? 

<h2 align='center'>======================================================================
If NOT well-formed SQL dump file then you must first do the following:
======================================================================</h2>

## 3. Removing lines that are not part of the actual content to be analysed

## 4. Splitting and naming tables and columns

<h2 align='center'>======================================================================
Additional functions
======================================================================</h2>

## * Function to read tables from wel-formed SQL database dump

In [11]:
# For well formed SQL database dumps!
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),', '', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                break
    sio.seek(0)
    return sio

## * Converter functions for formatting content of columns

In [12]:
def tolowercase(text):
    try:
        return text.lower()
    except AttributeError:
        return text 

In [13]:
def strip_singlequote(text):
    try:
        return text.strip('\'')
    except AttributeError:
        return text

In [14]:
def timestamp2datetime(timestamp):
    try:
        return  pd.to_datetime(timestamp,unit='s')
    except AttributeError:
        return timestamp 

In [15]:
def datetimestring2datetime(text):
    try:
        return  pd.to_datetime(text,format='%d-%m-%Y %H:%M')
    except AttributeError:
        return text 

In [16]:
def datetimeinvertedstring2datetime(text):
    try:
        return  pd.to_datetime(text,format='%Y-%m-%d %H:%M:%S')
    except AttributeError:
        return text 

In [17]:
def datetimeampm2datetime(text):
    try:
        return  pd.to_datetime(text,format='%d-%m-%Y %I:%M:%S %p')
    except AttributeError:
        return text 

In [18]:
def datetimemdyampm2datetime(text):
    try:
        return  pd.to_datetime(text,format='%m-%d-%Y %I:%M:%S %p')
    except AttributeError:
        return text 

In [19]:
def strip_singlequote_and_tolowercase(text):
    try:
        return text.strip('\'').lower()
    except AttributeError:
        return text 

In [20]:
def strip_singlequote_and_dateformat2datetime(text):
    try:
        return  pd.to_datetime(text.strip('\''),format='%d-%m-%Y %H:%M')
    except AttributeError:
        return text 

In [21]:
def strip_parenthesisandsemicolon(text):
    try:
        return text.replace(');',"")
    except AttributeError:
        return text 

In [22]:
def strip_insertintologs(text):
    try:
        return int(text.replace('INSERT INTO `logs` VALUES (',""))
    except AttributeError:
        return text 

<h2 align='center'>======================================================================
Adapting EACH existing table
======================================================================</h2>

## 6.1. Table: 'email_templates'

### How this table looks like without modification?

In [23]:
tablename='email_templates'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,name,subject,help,body
0,1 Registration Email Please verify your email ...,when Configuration->Registration Verification ...,244,244,NaN
1,2,Forgot Password Email,Password Reset,This template is used for retrieving lost user...,"<div align=""center"">\n<table width=""600"" cells..."
2,3 Welcome Mail From Admin You have been regist...,when user is added by administrator <div align...,244,244,NaN
3,4,Default Newsletter,Newsletter,This is a default newsletter template,"<div align=""center"">\n<table width=""600"" cells..."
4,5,Transaction Completed,Payment Completed,This template is used to notify administrator ...,"<div align=""center"">\n<table width=""600"" cells..."
5,6 Transaction Suspicious Suspicious Transactio...,244,244,NaN,NaN
6,7 Welcome Email Welcome This template is used ...,244,244,NaN,NaN
7,8,Membership Expire 7 days,Your membership will expire in 7 days,This template is used to remind user that memb...,"<div align=""center"">\n<table cellspacing=""5"" c..."
8,9,Membership Expired Today,Your membership has expired,This template is used to remind user that memb...,"<div align=""center"">\n<table width=""600"" cells..."
9,10,Contact Request,Contact Inquiry,This template is used to send default Contact ...,"<div align=""center"">\n<table width=""600"" cells..."


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type/converter: 
        o    
    - On the column name:
        o  
    - Split columns:
        o 
    - Add required columns:
        o 

## 6.2. Table: 'gateways'

### How this table looks like without modification?

In [24]:
tablename='gateways'

# Only displaying the results
pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,name,displayname,dir,demo,extra_txt,extra_txt2,extra_txt3,extra,extra2,extra3,is_recurring,active
0,1,paypal,PayPal,paypal,1,Paypal Email Address,Currency Code,Not in Use,r.evenge@aol.com,USD,NaN,1,1
1,2,moneybookers,MoneyBookers,moneybookers,1,MoneyBookers Email Address,Currency Code,Secret Passphrase,PandaHaxor@gmail.com,USD,pandahax,1,1


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: gateways
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o    
    - On the column name:
        o extra => email
    - Split columns:
        o 
    - Add required columns:
        o         

### Modifying table: 'gateways'

In [25]:
df_gateways = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_gateways.rename(columns = {'extra': 'email'},
                         inplace=True)

###Showing some lines after adapt the table
df_gateways.head(10)

,id,name,displayname,dir,demo,extra_txt,extra_txt2,extra_txt3,email,extra2,extra3,is_recurring,active
0,1,paypal,PayPal,paypal,1,Paypal Email Address,Currency Code,Not in Use,r.evenge@aol.com,USD,NaN,1,1
1,2,moneybookers,MoneyBookers,moneybookers,1,MoneyBookers Email Address,Currency Code,Secret Passphrase,PandaHaxor@gmail.com,USD,pandahax,1,1


## 6.3. Table: 'getshells'

### How this table looks like without modification?

In [26]:
tablename='getshells'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,URL,online,lastChecked
0,http://www.filbanken.nu/awstats/awstats/UDP.php,0,0
1,http://www.indianethicalhacker.blackapplehost....,0,0
2,http://www.indianethicalhacker.blackapplehost....,0,0
3,http://www.indianethicalhacker.blackapplehost....,0,0
4,http://163.178.170.74/webdav/greenshell.php,0,0
5,http://mapi.co.kr/zb41pl7/bbs/data/mapi_bbs/se...,0,0
6,http://indianethicalhacker.blackapplehost.com/...,0,0
7,http://theresahackforthat.webs.com/shell.php,0,0
8,http://www.ecofilms.gr/search_gr.asp,0,0
9,http://mmx1.webs.com/shell.php,0,0


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: 'webshells'
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o getshells => webshells
    - On the column type: 
    - On the column name:
        o URL => url
        o online => status
        o lastChecked => lastchecked
    - Add required columns:
        o attacktype = "GET"
    - Split columns:
        o 

### Modifying table: 'getshells'

In [27]:
### Loading the column converting columns with predefined functions
df_webshells = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_webshells.rename(columns = {'URL': 'url',
                               'online': 'status',
                               'lastChecked': 'lastchecked'},
                         inplace=True)

##Creating empty columns (with "" [for future string] or np.nan [for future float])
df_webshells['attacktype'] = "GET"

###Showing some lines after adapt the table
df_webshells.head(10)

,url,status,lastchecked,attacktype
0,http://www.filbanken.nu/awstats/awstats/UDP.php,0,0,GET
1,http://www.indianethicalhacker.blackapplehost....,0,0,GET
2,http://www.indianethicalhacker.blackapplehost....,0,0,GET
3,http://www.indianethicalhacker.blackapplehost....,0,0,GET
4,http://163.178.170.74/webdav/greenshell.php,0,0,GET
5,http://mapi.co.kr/zb41pl7/bbs/data/mapi_bbs/se...,0,0,GET
6,http://indianethicalhacker.blackapplehost.com/...,0,0,GET
7,http://theresahackforthat.webs.com/shell.php,0,0,GET
8,http://www.ecofilms.gr/search_gr.asp,0,0,GET
9,http://mmx1.webs.com/shell.php,0,0,GET


## 6.4. Table: 'logs'

### How this table looks like without modification?

In [28]:
tablename='logs'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,username,ip,time,port,date
0,Deceive,174.143.55.236,120,80,09-05-2011 06:57:38 am
1,Deceive,71.85.210.57,10,80,09-05-2011 06:58:47 am
2,Deceive,71.85.210.57,30,80,09-05-2011 07:02:59 am
3,Deceive,71.85.210.57,10,80,09-05-2011 07:05:59 am
4,Deceive,71.85.210.57,30,80,09-05-2011 07:10:56 am
5,Deceive,71.85.212.244,30,80,09-06-2011 12:55:50 am
6,MrdoPii,184.27.173.69,30,80,09-06-2011 01:08:06 am
7,MrdoPii,70.95.48.252,30,80,09-06-2011 01:17:43 am
8,MrdoPii,68.231.155.71,30,80,09-06-2011 01:22:05 am
9,MrdoPii,96.2.25.88,30,80,09-06-2011 01:24:32 am


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: attacks
#### Q3: Are there modifications required? 
    - On the table name: 
        o logs => attacks
    - On the column type: 
        o date => datetimemdyampm2datetime()  
    - On the column name:
        o ip => targetip
        o time => duration
    - Add required columns:
        o id
        o userid
        o targeturl
    - Split columns:
        o 

### Modifying table: 'logs'

In [29]:
### Loading the column converting columns with predefined functions
df_attacks = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False,
                       converters = {'date':datetimemdyampm2datetime})

###Changing names of columns
df_attacks.rename(columns = {'ip': 'targetip',
                                    'time': 'duration'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_attacks['id'] = np.nan
df_attacks['userid'] = np.nan
df_attacks['targeturl'] = ""

###Showing some lines after adapt the table
df_attacks.head(10)

,username,targetip,duration,port,date,id,userid,targeturl
0,Deceive,174.143.55.236,120,80,2011-09-05 06:57:38,NaN,NaN,
1,Deceive,71.85.210.57,10,80,2011-09-05 06:58:47,NaN,NaN,
2,Deceive,71.85.210.57,30,80,2011-09-05 07:02:59,NaN,NaN,
3,Deceive,71.85.210.57,10,80,2011-09-05 07:05:59,NaN,NaN,
4,Deceive,71.85.210.57,30,80,2011-09-05 07:10:56,NaN,NaN,
5,Deceive,71.85.212.244,30,80,2011-09-06 00:55:50,NaN,NaN,
6,MrdoPii,184.27.173.69,30,80,2011-09-06 01:08:06,NaN,NaN,
7,MrdoPii,70.95.48.252,30,80,2011-09-06 01:17:43,NaN,NaN,
8,MrdoPii,68.231.155.71,30,80,2011-09-06 01:22:05,NaN,NaN,
9,MrdoPii,96.2.25.88,30,80,2011-09-06 01:24:32,NaN,NaN,


## 6.5 Table: 'memberships'

### How this table looks like without modification?

In [30]:
tablename='memberships'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,title,description,price,days,period,trial,recurring,private,active
0,11,1 Day,NaN,1.0,1,D,0,1,0,1
1,12,Lifetime,NaN,20.0,2,Y,0,1,0,1
2,13,1 Month,NaN,9.0,1,M,0,1,0,1
3,14,3 Month,NaN,13.0,3,M,0,1,0,1


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: plans
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o memberships => plans
    - On the column type: 
    - On the column name:
        o id => planid
        o title => planname 
        o description => plandescr
        o recurring => concurrecy
    - Add required columns:
        o maxboottime 
    - Split columns:
        o 

### Modifying table: 'memberships'

In [31]:
### Loading the column converting columns with predefined functions
df_plans = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_plans.rename(columns = {'id': 'planid',
                                    'title': 'planname',
                           'description':'plandescr',
                          'recurring':'concurrency'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_plans['maxboottime'] = np.nan

###Showing some lines after adapt the table
df_plans.head(10)

,planid,planname,plandescr,price,days,period,trial,concurrency,private,active,maxboottime
0,11,1 Day,NaN,1.0,1,D,0,1,0,1,NaN
1,12,Lifetime,NaN,20.0,2,Y,0,1,0,1,NaN
2,13,1 Month,NaN,9.0,1,M,0,1,0,1,NaN
3,14,3 Month,NaN,13.0,3,M,0,1,0,1,NaN


## 6.6. Table: 'news'

### How this table looks like without modification?

In [32]:
tablename='news'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,title,body,author,created,active
0,1,Welcome to our Client Area!,<p>We are pleased to announce the new release ...,Yahh Daddy Panda,0000-00-00,0.0
1,2 Please Vouch / Donate I am currently getting...,and with that being said I will not be able to...,Panda,2012-05-18,1,NaN


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o    
    - On the column name:
        o  
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.7. Table: 'postshells'

### How this table looks like without modification?

In [33]:
tablename='postshells'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,URL,online,lastChecked
0,http://feeds2.feedburner.com/alivingtruth/DRwH,0,0
1,http://d-dos.50webs.com/ddos.php,0,0
2,http://belakshell.50webs.com/index.php,0,0
3,http://reksa.indonesianhacker.com/php_dos/,0,0
4,http://yogeshmehra1987.110mb.com/index.php,0,0
5,http://kala13.110mb.com/dos/,0,0
6,http://urmybest.110mb.com/phpdos.php,0,0
7,http://urmybest.110mb.com/phpdos.php,0,0
8,http://yogeshmehra1987.110mb.com/index.php,0,0
9,http://firman-mannte.blogspot.com/2011_04_01_a...,0,0


#### Q1: Is this table different from other previous loaded table? YES, although very similar APPEND!!!!
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: webshells
#### Q3: Are there modifications required? 
    - On the table name: 
        o postshells => webshells_temp THEN append with webshell
    - On the column type: 
        o    
    - On the column name:
        o URL => url
        o online => status
        o lastChecked => lastchecked 
    - Add required columns:
        o id
        o attacktype = 'POST'
    - Split columns:
        o 

### Modifying table: 'postshells'

In [34]:
### Loading the column converting columns with predefined functions
df_webshells_temp = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_webshells_temp.rename(columns = {'URL': 'url',
                               'online': 'status',
                               'lastChecked': 'lastchecked'},
                         inplace=True)

##Creating empty columns (with "" [for future string] or np.nan [for future float])
df_webshells_temp['attacktype'] = "POST"

###Showing some lines after adapt the table
df_webshells_temp.head(10)

,url,status,lastchecked,attacktype
0,http://feeds2.feedburner.com/alivingtruth/DRwH,0,0,POST
1,http://d-dos.50webs.com/ddos.php,0,0,POST
2,http://belakshell.50webs.com/index.php,0,0,POST
3,http://reksa.indonesianhacker.com/php_dos/,0,0,POST
4,http://yogeshmehra1987.110mb.com/index.php,0,0,POST
5,http://kala13.110mb.com/dos/,0,0,POST
6,http://urmybest.110mb.com/phpdos.php,0,0,POST
7,http://urmybest.110mb.com/phpdos.php,0,0,POST
8,http://yogeshmehra1987.110mb.com/index.php,0,0,POST
9,http://firman-mannte.blogspot.com/2011_04_01_a...,0,0,POST


In [35]:
df_webshells = df_webshells.append(df_webshells_temp)

## 6.8. Table: 'settings'

### How this table looks like without modification?

In [36]:
tablename='settings'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,site_name,site_email,site_url,reg_allowed,user_limit,reg_verify,notify_admin,auto_verify,user_perpage,thumb_w,thumb_h,backup,currency,cur_symbol,mailer,smtp_host,smtp_user,smtp_pass,smtp_port,version
0,Panda Booter | Webbased Edition,PandaHaxor@Gmail.com,http://46.166.148.212/~pandabo1/,1,30,0,1,1,10,100,100,NaN,USD,$,PHP,NaN,NaN,NaN,0,2.0


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: settings
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o    
    - On the column name:
        o site_url => url 
        o site_name =>sitename
        o site_email => siteemail
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'settings'

In [37]:
### Loading the column converting columns with predefined functions
df_settings = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_settings.rename(columns = {'site_url': 'url',
                              'site_name':'sitename',
                              'site_email': 'siteemail'},
                         inplace=True)

###Showing some lines after adapt the table
df_settings.head(10)

,sitename,siteemail,url,reg_allowed,user_limit,reg_verify,notify_admin,auto_verify,user_perpage,thumb_w,thumb_h,backup,currency,cur_symbol,mailer,smtp_host,smtp_user,smtp_pass,smtp_port,version
0,Panda Booter | Webbased Edition,PandaHaxor@Gmail.com,http://46.166.148.212/~pandabo1/,1,30,0,1,1,10,100,100,NaN,USD,$,PHP,NaN,NaN,NaN,0,2.0


## 6.9. Table: 'users'

### How this table looks like without modification?

In [38]:
tablename='users'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,username,password,membership_id,mem_expire,trial_used,email,fname,lname,token,newsletter,userlevel,created,lastlogin,lastip,avatar,active,myAttacks
0,15,Assassinator980,d55bcc6b455abc01e8939d970168101a16f11898,12,2014-05-17 11:19:19,0,zingcraft@hotmail.com,Liam,Edwards,e936c4a8e5b6230ed915a46a0009fe5e78ce8693,0,1,2012-05-17 10:47:23,2012-05-20 10:44:11,58.106.28.31,NaN,y,12.0
1,17,Failedattempt,56c37153fc1d06d3a198a39f82bdf8e17023669f,12,2014-05-17 11:30:54,0,jolliffboy98@hotmail.com,Gary,J,0,0,1,2012-05-17 11:23:31,2012-05-18 12:56:55,184.10.176.32,NaN,y,2.0
2,20,I nfizr,d361c77d5fc37c45b35f1883d2b2b2127fcd3a25,12,2014-05-17 14:36:56,0,mauricekleijwegt@hotmail.com,maurice,kleijwegt,0,0,1,2012-05-17 14:32:24,2012-05-17 17:27:59,84.84.176.106,IMG_1DD90F-1340AD-B16D6E-2D199F-54CF7B-C84F1F.jpg,y,2.0
3,12,Cudd,7325e5b55758a26264dd7615be4157363bf46f39,11,2012-05-18 10:33:21,0,allano123@hotmail.dk,Joseph,Mariuana,f479f0b33757950b0c002e44bbee9ce411a6120c,0,1,2012-05-17 10:32:08,2012-05-17 10:36:44,188.177.190.110,NaN,y,1.0
4,11,Panda,0716b9029d0818cbabd7c69aa55d01c877982b54,12,2014-05-17 10:39:12,0,pandahaxor@gmail.com,Panda,Admin,3d6db250cb14beeae5648bc57b2479751471d660,1,9,2012-05-17 10:03:10,2012-05-20 12:13:53,93.36.195.32,IMG_AF98D6-320F74-69A004-8C09A0-F13C43-CF0B26.png,y,6.0
5,18,Green,5b0f274aa43672b66d5587baf63cf6d9354956bc,12,2014-05-17 11:30:40,0,Presee@hotmail.co.uk,Mike,Rw,0,0,1,2012-05-17 11:24:55,2012-05-18 22:18:09,86.133.207.153,NaN,y,5.0
6,19,DutchConquer,da7a3c650e218272c37e387104a571a8b374bb6a,0,0000-00-00 00:00:00,0,dutchconquerhf@hotmail.com,Martin,L,0,0,1,2012-05-17 13:44:15,0000-00-00 00:00:00,0,NaN,y,NaN
7,21,wisdom,27d769673277ad1659a3342e8c92ca84a95c4166,12,2014-05-17 18:21:38,0,b3t4_01@yahoo.com,budi,santoso,0,0,1,2012-05-17 14:37:43,2012-05-17 14:40:44,110.138.209.45,NaN,y,NaN
8,22,Booti23,55e663cb3685e142cdafe723776ded5a9d84b140,12,2014-05-17 18:07:51,0,tara.od@hotmail.com,Tara,Power,0,0,1,2012-05-17 15:26:55,2012-05-19 01:02:53,212.7.208.75,NaN,y,8.0
9,24,MarkFortune,83a4a122db5812134055dcccfbdf31bf5dacfe62,12,2014-05-18 18:27:55,0,Marksfortune4@gmail.com,Mark,Fortune,0,0,9,2012-05-17 18:20:22,2012-05-19 03:34:45,72.91.100.167,NaN,y,3.0


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: users
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o mem_expire => datetimestringinverted2datetime 
        o lastlogin => datetimestringinverted2datetime
    - On the column name:
        o id => userid
        o email => useremail 
        o mem_expire => expire
        o membership_id => planid
    - Add required columns:
        o 
    - Split columns:
        o 
        
    - Split columns to create new table:
    df_logins = df_users[['userid','username','lastlogin','lastip']]
    then change:
    lastlogin => date
    lastip => userip

### Modifying table: 'users'

In [39]:
### Loading the column converting columns with predefined functions
df_users = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_users.rename(columns = {'id': 'userid',
                          'email': 'useremail',
                          'mem_expire': 'expire',
                          'membership_id': 'planid'},
                         inplace=True)

###Showing some lines after adapt the table
df_users.head(10)

,userid,username,password,planid,expire,trial_used,useremail,fname,lname,token,newsletter,userlevel,created,lastlogin,lastip,avatar,active,myAttacks
0,15,Assassinator980,d55bcc6b455abc01e8939d970168101a16f11898,12,2014-05-17 11:19:19,0,zingcraft@hotmail.com,Liam,Edwards,e936c4a8e5b6230ed915a46a0009fe5e78ce8693,0,1,2012-05-17 10:47:23,2012-05-20 10:44:11,58.106.28.31,NaN,y,12.0
1,17,Failedattempt,56c37153fc1d06d3a198a39f82bdf8e17023669f,12,2014-05-17 11:30:54,0,jolliffboy98@hotmail.com,Gary,J,0,0,1,2012-05-17 11:23:31,2012-05-18 12:56:55,184.10.176.32,NaN,y,2.0
2,20,I nfizr,d361c77d5fc37c45b35f1883d2b2b2127fcd3a25,12,2014-05-17 14:36:56,0,mauricekleijwegt@hotmail.com,maurice,kleijwegt,0,0,1,2012-05-17 14:32:24,2012-05-17 17:27:59,84.84.176.106,IMG_1DD90F-1340AD-B16D6E-2D199F-54CF7B-C84F1F.jpg,y,2.0
3,12,Cudd,7325e5b55758a26264dd7615be4157363bf46f39,11,2012-05-18 10:33:21,0,allano123@hotmail.dk,Joseph,Mariuana,f479f0b33757950b0c002e44bbee9ce411a6120c,0,1,2012-05-17 10:32:08,2012-05-17 10:36:44,188.177.190.110,NaN,y,1.0
4,11,Panda,0716b9029d0818cbabd7c69aa55d01c877982b54,12,2014-05-17 10:39:12,0,pandahaxor@gmail.com,Panda,Admin,3d6db250cb14beeae5648bc57b2479751471d660,1,9,2012-05-17 10:03:10,2012-05-20 12:13:53,93.36.195.32,IMG_AF98D6-320F74-69A004-8C09A0-F13C43-CF0B26.png,y,6.0
5,18,Green,5b0f274aa43672b66d5587baf63cf6d9354956bc,12,2014-05-17 11:30:40,0,Presee@hotmail.co.uk,Mike,Rw,0,0,1,2012-05-17 11:24:55,2012-05-18 22:18:09,86.133.207.153,NaN,y,5.0
6,19,DutchConquer,da7a3c650e218272c37e387104a571a8b374bb6a,0,0000-00-00 00:00:00,0,dutchconquerhf@hotmail.com,Martin,L,0,0,1,2012-05-17 13:44:15,0000-00-00 00:00:00,0,NaN,y,NaN
7,21,wisdom,27d769673277ad1659a3342e8c92ca84a95c4166,12,2014-05-17 18:21:38,0,b3t4_01@yahoo.com,budi,santoso,0,0,1,2012-05-17 14:37:43,2012-05-17 14:40:44,110.138.209.45,NaN,y,NaN
8,22,Booti23,55e663cb3685e142cdafe723776ded5a9d84b140,12,2014-05-17 18:07:51,0,tara.od@hotmail.com,Tara,Power,0,0,1,2012-05-17 15:26:55,2012-05-19 01:02:53,212.7.208.75,NaN,y,8.0
9,24,MarkFortune,83a4a122db5812134055dcccfbdf31bf5dacfe62,12,2014-05-18 18:27:55,0,Marksfortune4@gmail.com,Mark,Fortune,0,0,9,2012-05-17 18:20:22,2012-05-19 03:34:45,72.91.100.167,NaN,y,3.0


## Creating table logins from table users

In [42]:
df_logins = df_users[['userid','username','lastlogin','lastip']]

df_logins.rename(columns = {'lastlogin': 'date',
                          'lastip': 'userip'},
                         inplace=True)

df_logins.head(10)

/usr/local/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,userid,username,date,userip
0,15,Assassinator980,2012-05-20 10:44:11,58.106.28.31
1,17,Failedattempt,2012-05-18 12:56:55,184.10.176.32
2,20,I nfizr,2012-05-17 17:27:59,84.84.176.106
3,12,Cudd,2012-05-17 10:36:44,188.177.190.110
4,11,Panda,2012-05-20 12:13:53,93.36.195.32
5,18,Green,2012-05-18 22:18:09,86.133.207.153
6,19,DutchConquer,0000-00-00 00:00:00,0
7,21,wisdom,2012-05-17 14:40:44,110.138.209.45
8,22,Booti23,2012-05-19 01:02:53,212.7.208.75
9,24,MarkFortune,2012-05-19 03:34:45,72.91.100.167


<h2 align='center'>======================================================================
<br>Final step of the manual part
======================================================================</h2>

## 7. Adding missing tables accordingly to our generic Booter database 

In [43]:
df_attacks= pd.DataFrame(columns=['id','userid','username','targetip','targeturl','duration','port','type','date'])
df_blacklist=pd.DataFrame(columns=['id','ip','note'])
df_friendsenemies=pd.DataFrame(columns=['id','ip','note','userid','type'])
# df_gateways=pd.DataFrame(columns=['email'])
# df_logins=pd.DataFrame(columns=['id','userid','username','userip','date'])
df_payments=pd.DataFrame(columns=['id','userid','username','amountpaid','paymentemail','planid','tid','date'])
df_plans=pd.DataFrame(columns=['planid','planname','plandescr','price','maxboottime','concurrency'])
df_servers=pd.DataFrame(columns=['id','ip'])
# df_settings=pd.DataFrame(columns=['url','sitename','siteemail'])
# df_users=pd.DataFrame(columns=['userid','username','useremail','password','expire','plan'])      
# df_webshells=pd.DataFrame(columns=['id','url','status','lastchecked','attacktype'])

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 2: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

### * Depending of the size of the data, this part can take HOURS. I tested for both small and big datasets and it worked. Be pacient. This will pay-off!

## * Libraries needed to retrieve information from external databases

In [ ]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

## * Function to enrich IP addresseswith AS information and country 

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

## * Function to check if an IP address was Tor node in a given moment

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## * Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## * Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 2.1.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('enrichments/logins_iptoasn_out')== False):
    logins_iptoasn_in = open('enrichments/logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('enrichments/logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

df_logins_extended= pd.merge(df_logins,
                              df_logins_iptoasn,
                              how = 'left',
                              left_on = 'userip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_logins_extended.rename(columns={'asn':'srcasn', 
                                   'ip':'srcip', 
                                   'bgp_prefix':'srcbgp_prefix', 
                                   'country':'srccountry' ,
                                   'registry':'srcregistry',
                                   'info_date':'srcinfo_date',
                                   'info_request':'srcinfo_request'},
                         inplace=True)


df_logins_extended.head(5)

## 2.1.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('enrichments/attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('enrichments/attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('enrichments/attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_attacks_extended= pd.merge(df_attacks,
                              df_attacks_iptoasn,
                              how = 'left',
                              left_on = 'targetip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_attacks_extended.rename(columns={'asn':'targetasn', 
                                   'ip_y':'targetip', 
                                   'bgp_prefix':'targetbgp_prefix', 
                                   'country_y':'targetcountry' ,
                                   'registry':'targetregistry',
                                   'info_date':'targetinfo_date',
                                   'info_request':'targetinfo_request'},
                         inplace=True)
df_attacks_extended.head(5)

## 2.1.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('enrichments/friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('enrichments/friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('enrichments/friendsenemies_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_friendsenemies_extended= pd.merge(df_friendsenemies,
                              df_friendsenemies_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_friendsenemies_extended.rename(columns={'asn':'friendsenemiesasn', 
                                   'ip':'friendsenemiesip', 
                                   'bgp_prefix':'friendsenemiesbgp_prefix', 
                                   'country':'friendsenemiescountry' ,
                                   'registry':'friendsenemiesregistry',
                                   'info_date':'friendsenemiesinfo_date',
                                   'info_request':'friendsenemiesinfo_request',
                                   'as_name': 'friendsenemiesas_name'},
                         inplace=True)

## 2.1.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('enrichments/blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('enrichments/blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('enrichments/blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_blacklist_extended= pd.merge(df_blacklist,
                              df_blacklist_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_blacklist_extended.rename(columns={'asn':'blacklistasn', 
                                   'ip':'blacklistip', 
                                   'bgp_prefix':'blacklistbgp_prefix', 
                                   'country':'blacklistcountry' ,
                                   'registry':'blacklistregistry',
                                   'info_date':'blacklistinfo_date',
                                   'info_request':'blacklistinfo_request',
                                   'as_name': 'blacklistas_name'},
                         inplace=True)
df_blacklist_extended.head(5)

## 2.2.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if len(df_logins['userip'].unique())<1200:
    if (os.path.exists('enrichments/logins_torcheck')== False):
        print("Note: it can take a while to finish...")
        logins_torcheck = open('logins_torcheck', 'w+')
        for i in df_logins['userip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=logins_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            logins_torcheck.flush()

        logins_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.")
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_logins_torcheck = pd.read_csv('enrichments/logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

## 2.2.2. Check if IP was a TOR node for table: attacks

In [ ]:
if len(df_attacks['targetip'].unique())<1200:
    if (os.path.exists('enrichments/attacks_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_attacks['targetip'].unique())*3,"seconds (in the worst case).")

        attacks_torcheck = open('attacks_torcheck', 'w+')

        for i in df_attacks['targetip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=attacks_torcheck)
            print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            attacks_torcheck.flush()

        attacks_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_attacks_torcheck = pd.read_csv('enrichments/attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

## 2.2.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if len(df_friendsenemies['ip'].unique()) <1200:
    if (os.path.exists('enrichments/friendsenemies_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_friendsenemies['ip'].unique())*3,"seconds (in the worst case).")

        friendsenemies_torcheck = open('enrichments/friendsenemies_torcheck', 'w+')

        for i in df_friendsenemies['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=friendsenemies_torcheck)
        #     print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            friendsenemies_torcheck.flush()

        friendsenemies_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('enrichments/friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## 2.2.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if len(df_blacklist['ip'].unique()) < 1200:
    if (os.path.exists('enrichments/blacklist_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_blacklist['ip'].unique())*3,"seconds (in the worst case).")

        blacklist_torcheck = open('enrichments/blacklist_torcheck', 'w+')

        for i in df_blacklist['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=blacklist_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            blacklist_torcheck.flush()

        blacklist_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_blacklist_torcheck = pd.read_csv('enrichments/blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]

In [ ]:
#This is the TOTAL number records to be checks!!!!
len(df_attacks)

In [ ]:
df_attacks['nearestlogin']=""
df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])
    
if len(df_attacks)>0 and len(df_logins)>0:
    #When was the last login of the user that performed attacks
    df_attacks['nearestlogin']=""
    df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])

    for index, row in df_attacks.head(100).iterrows():
        intermediate_df= df_logins[df_logins['username']==row['username']]
        nearestlogindate= nearestDate(row['date'],intermediate_df['date'])
        df_attacks.set_value(index, 'nearestlogin', nearestlogindate)

        #DEBUGGING
        if index % 1000 == 0:
            print(index,": +1000 records analysed")

In [ ]:
if len(df_attacks['nearestlogin'].value_counts()) >1:
    df_attacks_and_logins = pd.merge(df_attacks_extended,
                                     df_logins_extended,
                                     how = 'left',
                                     left_on = ['username','nearestlogin'],
                                     right_on = ['username','date'])
else:
    df_attacks_and_logins=""

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 3: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## * Libraries that I use to plot figs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

import seaborn as sns
import itertools

plt.style.use('seaborn-muted')
# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## A.1. Number of records per table (part of the generic Booter database schema)

In [ ]:
len(df_users),len(df_logins),len(df_attacks),len(df_payments),len(df_settings),len(df_gateways), len(df_friendsenemies),len(df_blacklist),len(df_webshells),len(df_servers),len(df_plans)

## A.2. Number of users, customers, attackers, and their intersection

In [ ]:
if len(set(df_users['userid'].unique())) > 1:
    users_set=set(df_users['userid'].unique())
else:
    users_set=set(df_users['username'].unique())
    
if len(set(df_logins['userid'].unique())) > 1:
    userslogin_set=set(df_logins['userid'].unique())
else:
    userslogin_set=set(df_logins['username'].unique())

if len(set(df_payments['userid'].unique())) > 1:
    customers_set=set(df_payments['userid'].unique())
else:
    customers_set=set(df_payments['username'].unique())

if len(set(df_attacks['userid'].unique())) > 1:
    attackers_set=set(df_attacks['userid'].unique())
else:
    attackers_set=set(df_attacks['username'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))
intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
len(users_set),len(userslogin_set),len(customers_set),len(attackers_set)

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=14)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## A.3. Distribution of login times per user

In [ ]:
if len(df_logins['userid'].value_counts()) > 0:
    num_distinct_logins_per_user=df_logins['userid'].value_counts()
else:
    num_distinct_logins_per_user=df_logins['username'].value_counts()

freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Login Times by Users:', fontsize=14, y=1.05,x=0.35)
    
    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    ax1.set_title("")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', 
                                       labels=None, 
                                       legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    
    
    
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_logins_per_user.values)/(freq_distinct_logins_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_logins_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/login_times.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.3. Number of Users that Login via TOR

In [ ]:
len(df_logins_torcheck[df_logins_torcheck['tor']==True]['userip'].value_counts())

## 5.4. Number of Distinct IP addresses by Users

In [ ]:
if len(df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()) >0:
    num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
else:
    num_distinct_ips_per_user=df_logins.groupby(['username','userip']).size().reset_index()['username'].value_counts()
    
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Distinct IP address used by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie',
                                    labels=None,legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_ips_per_user.values)/(freq_distinct_ips_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_ips_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/num_ips_by_users.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.5. Number of Payments by Users

In [ ]:
if len(df_payments['userid'].value_counts())>0:
    num_distinct_payments_per_user=df_payments['userid'].value_counts()
else:
    num_distinct_payments_per_user=df_payments['username'].value_counts()

freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Number of Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', 
                                         labels=None,legend=False,
                                         startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_per_user.values)/(freq_distinct_payments_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/payments_distribution.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.6. Total Amount of Money Earned

In [ ]:
if len(df_payments['amountpaid']) >0:
    total_earned=df_payments['amountpaid'].values.sum()
    'US$ {:,.2f}'.format(float(total_earned))
else:
    print("Unfortunately, there is no data available!")

## 5.7. Amount of Money Paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Money Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', 
                                               labels=None,legend=False,
                                               startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_money_per_user.values)/(freq_distinct_payments_money_per_user.values.sum())
    labels = ['${0:1.2f} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_money_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.6, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/distribution_amount_paid.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.8. Countries from where Users Access Booters

In [ ]:
logins_country_distribution_sorted = df_logins_iptoasn['country'].value_counts(sort=True,ascending=True)
logins_country_distribution = df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Users Accessing from Countries:', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', 
                                     labels=None,legend=False,
                                     startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*logins_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(logins_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/user_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of Blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()
blacklist_country_distribution_sorted=df_blacklist_iptoasn['country'].value_counts(sort=True,ascending=True)

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of blacklisted IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', 
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*blacklist_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(blacklist_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/blacklist_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.10. Countries of Target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of target IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', 
                                      labels=None,legend=False,
                                      startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*attacks_country_distribution.values)/(attacks_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(attacks_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/attack_countries_distribution.eps',bbox_inches='tight', format='eps', dpi=1200) 
else:
    print("Unfortunately, there is no data available!")

## 5.12. Attacks on Same Target

In [ ]:
num_attacks_on_sametarget=df_attacks['targetip'].value_counts()

freq_num_attacks_on_sametarget=num_attacks_on_sametarget.value_counts()
cum_num_attacks_on_sametarget = np.linspace(0.,1.,len(num_attacks_on_sametarget))
cdf_num_attacks_on_sametarget = pd.Series(cum_num_attacks_on_sametarget, index=num_attacks_on_sametarget.sort_values())

In [ ]:
if len(num_attacks_on_sametarget)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Attacks on Same Targets:', fontsize=14, y=1.05,x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_num_attacks_on_sametarget.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_num_attacks_on_sametarget.plot(ax=ax2,kind='pie',
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_num_attacks_on_sametarget.values)/(freq_num_attacks_on_sametarget.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_num_attacks_on_sametarget.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()

    fig.show()
    fig.savefig('figs/attacks_on_same_target.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.11. Attacks per day (timeseries) 

In [ ]:
if len(df_attacks)>0:
    attack_timeseries=df_attacks.set_index(['date']).groupby(pd.TimeGrouper(freq='D')).agg(['count'])['action']
    attack_mean_perday=attack_timeseries.mean()
    attack_median_perday=attack_timeseries.median()
else:
    attack_timeseries=""

In [ ]:
if len(attack_timeseries)>0:
    fig = plt.figure(figsize=(6,3))

    ax1 = plt.subplot2grid((1,1), (0,0), rowspan=2)
    attack_timeseries.plot(ax=ax1,
                           lw=2,
                           legend=False,
    #                        style='--'
                          )

    # X and Y Labels and Ticks
    ax1.set_xlabel("Time (bin=day)")
    ax1.set_ylabel("# Attacks")

    ax1.annotate(str(int(attack_median_perday[0]))+' (median)', 
                 (min(df_attacks['date']), attack_median_perday),
                 xytext=(350, -1), 
                 textcoords='offset points',
                 color='black', 
                 arrowprops=dict(arrowstyle='-|>',
                                 color='black'))
    fig.savefig('figs/attacks_timeseries.eps', bbox_inches='tight',format='eps', dpi=1200)

else:
    print("Unfortunately, there is no data available!")

## 5.12. Time Between Logins and Attacks

In [ ]:
if len(df_attacks_and_logins)>0:
    print("redo")
else:
    print("Unfortunately, there is no data available!")

## 5.13. Who attack whom? (users on the country level)

In [ ]:
if len(df_attacks_extended)>0 and len(df_logins_extended)>0:
    merged_attacks_logins = pd.merge(df_attacks_extended,
                                  df_logins_extended,
                                  how = 'left',
                                  left_on = 'date',
                                  right_on = 'date')[['targetcountry','srccountry']]

    who_against_whom = merged_attacks_logins.groupby(['targetcountry','srccountry'])\
                            .size()\
                            .reset_index()\
                            .pivot('srccountry','targetcountry',0)
else:
    who_against_whom =""

In [ ]:
if len(who_against_whom)>0:
    fig = plt.figure(figsize=(8,12))
    fig.suptitle('Countries of Target IPs', fontsize=14, y=.92)

    ax1 = plt.subplot2grid((1,1), (0,0))
    sns.set()
    sns.heatmap(who_against_whom,
                ax=ax1,
#                 cmap="YlGnBu",
    #             linewidths=.5,
    #             annot=True
                )

    ax1.set_ylabel("Source Country")
    ax1.set_xlabel("Attack Target Country")

    fig.show()
    fig.savefig('figs/who_attack_whom.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

<h1 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [ ]:
from ipywidgets import widgets, interact, Dropdown,HTML
from IPython.display import display,clear_output

In [ ]:
def search_by_userid_submit(sender):
    clear_output()
    print("Searching by id =",search_by_userid.value)
    
    if len(df_users[df_users['userid']== int(search_by_userid.value)])>0:
        display(df_users[df_logins['userid']== int(search_by_userid.value)])
    
    if len(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])>0:
        display(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])
    
    if len(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])>0:
        display(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])
    
    if len(df_payments[df_payments['userid']== int(search_by_userid.value)])>0:
        display(df_payments[df_payments['userid']== int(search_by_userid.value)])

def search_by_username_submit(sender):
    clear_output()
    print("Searching by username =",search_by_username.value,"\n")
    
    if len(df_users[df_users['username']== search_by_username.value])>0:
        print("Table df_users:")
        display(df_users[df_users['username']== search_by_username.value])
    
    if len(df_logins_extended[df_logins_extended['username']== search_by_username.value])>0:
        print("Table df_logins:")
        display(df_logins_extended[df_logins_extended['username']== search_by_username.value])

    if len(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])>0:
        print("Table df_attacks:")
        display(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])
    
    if len(df_payments[df_payments['username']== search_by_username.value])>0:
        print("Table df_payments:")
        display(df_payments[df_payments['username']== search_by_username.value])
        
def search_by_ip_submit(sender):
    clear_output()
    print("Searching by IP address =",str(search_by_ip.value),"\n")
    
    if len(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])>0:
        print("Table df_logins (as ATTACKER:")
        display(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])
     
    if len(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])  
      
       
def search_by_asn_submit(sender):
    clear_output()
    print("Searching by Autonomous System Number (ASN) =",search_by_asn.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])
    
    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

        
country_list=pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv',delimiter=",",error_bad_lines=False)
def search_by_country_submit():
    country_code=country_list[country_list['Name']==search_by_country.value]['Code'].values[0]
    clear_output()
    print("Searching by Country =",search_by_country.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srccountry']== country_code])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srccountry']== country_code])
    
    if len(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])
    

In [ ]:
display(HTML('<h3>*Search by (only) one field per time:</h3>'))

search_by_username = widgets.Text(description="username")
display(search_by_username)
search_by_username.on_submit(search_by_username_submit)

search_by_userid = widgets.Text(description="userid")
display(search_by_userid)
search_by_userid.on_submit(search_by_userid_submit)

search_by_ip = widgets.Text(description="IP")
display(search_by_ip)
search_by_ip.on_submit(search_by_ip_submit)

search_by_asn = widgets.Text(description="ASN")
display(search_by_asn)
search_by_asn.on_submit(search_by_asn_submit)

search_by_country = Dropdown(description="Country", options=country_list['Name'].tolist())
search_by_country.on_trait_change(search_by_country_submit, name="value")
display(search_by_country)